# Complete Exploratory Analysis, Model Selection and Hyper-Parameter Optimization of the Flu-Shot Prediction dataset

# Table of Contents
* [Introduction](#intro)
* [Preliminary analysis & Data Processing](#prelim)
* [Exploratory Data Analysis and Visualizations](#eda)
* [Feature Selection, Extraction & Engineering](#fe)
* [Data Splitting and Test data processing](#split)
* [Analyzing Several Models on the Dataset](#models)
    - [Ensemble of Models](#ensemble)
    - [Models with Hyper Parameter Optimization (GSCV & RSCV)](#hpo)
* [Final Notes & Submission](#final)

# 1. Introduction <a id="intro"></a> 

> The Objective is to perform a multilabel classification of whether a person will/will not take the:
  >  1. H1N1 vaccine 
  >  2. Seasonal Flu vaccine

### The data can be found here: https://www.kaggle.com/darkknight98/flu-shot-prediction

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
### Necessary imports ###

import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
import sklearn as sk
import matplotlib as mpl
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.metrics import roc_auc_score,f1_score,accuracy_score
from sklearn.model_selection import train_test_split,StratifiedKFold,cross_val_score
from sklearn.preprocessing import LabelEncoder,OrdinalEncoder
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.ensemble import RandomForestClassifier as RF
import lightgbm as lgb
from xgboost import XGBClassifier
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.calibration import calibration_curve
from sklearn.preprocessing import PolynomialFeatures
from catboost import Pool,CatBoostClassifier
from xgboost import XGBClassifier
from catboost import CatBoostRegressor
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn import svm, model_selection,tree, linear_model, neighbors, naive_bayes, ensemble 
from sklearn import discriminant_analysis, gaussian_process

# 2. Preliminary analysis & Data Processing <a id="prelim"></a> 

Reading the data

In [ ]:
X = pd.read_csv(r'/kaggle/input/flu-shot-prediction/training_set_features.csv')
Y = pd.read_csv(r'/kaggle/input/flu-shot-prediction/training_set_labels.csv')

In [ ]:
print('Shape of X: ',X.shape)
print('Shape of Y: ',Y.shape)

> We see that there are 36 columns/features in the training set 

> There are 3 columns in the target file(ground truth) which include respondent_id, h1n1_vaccine,seasonal_vaccine

In [ ]:
X.info()

In [ ]:
X.head()

Several features including a persons 'behavior' and his/her 'opinion' about the vaccine, marital status, geographic location all are found, so the data consists a significantly large amount of info

In [ ]:
X.columns

In [ ]:
### Joining the X & Y into a single dataframe ###
Z = Y
Z.drop('respondent_id',axis = 1,inplace = True)
frames = [X,Z]
data = pd.concat(frames,axis=1)

Checking out the number of NULLs/NaNs

In [ ]:
data.isnull().sum()

Columns **'health_insurance', 'employment_industry' , 'employment_occupation'** contain too many NULLs

In [ ]:
# close to half the values of these columns are NULL so discarding
data.drop(['health_insurance','employment_industry','employment_occupation'],axis=1,inplace = True)

# 3. Exploratory Data Analysis and Visualizations <a id="eda"></a> 

### Firstly Analyzing the number of people who took each vaccine

In [ ]:
sns.factorplot('h1n1_vaccine',data=Y,kind='count',size = 3.5)
sns.factorplot('seasonal_vaccine',data=Y,kind='count',size = 3.5)

Considerably larger number of people have chosen to take the seasonal flu vaccine compared to those who took the H1N1 vaccine

### Next Finding the categorical features present in the data 

In [ ]:
str_cols = data.select_dtypes(include = 'object').columns
data[str_cols].head()

There are 10 categorical features!

### Populating the NULLs/NaNs with suitable substitutes

P.S: All Categorical features which are NULL are substituted with the mode of data, and the Numeric features with NULLs are substituted with the means

In [ ]:
# All the NULL values are populated with the mode
for col in data.columns:
    if data[col].isnull().sum() and data[col].dtypes != 'object':
        data[col].loc[(X[col].isnull())] = data[col].median()
for col in data.columns:
    if data[col].isnull().sum() and data[col].dtypes == 'object':
        data[col].loc[(data[col].isnull())] = data[col].mode().max()

Sanity Check below!!

In [ ]:
data.isnull().sum()

### Categorical Features are encoded using Label Encodings

P.S: Please consider trying out Mean Encodings, One Hot Encoding or Ordinal Encodings and notice the differences

In [ ]:
LE = LabelEncoder()
for col in str_cols:
    data[col] = LE.fit_transform(data[col]) # Converts to int64

Sanity Check again!

In [ ]:
data[str_cols].head()

### Plotting Correlation Maps with the set of Encoded and Null Populated Features

In [ ]:
corr = data.corr()
g = sns.heatmap(corr,  vmax=.3, center=0,
            square=True, linewidths=1, cbar_kws={"shrink": .5}, annot=True, fmt='.2f', cmap='Spectral')
sns.despine()
g.figure.set_size_inches(30,25)
    
plt.show()

*  High Positive correlations between the 'behavioral_features' is noted, some of them may be redundant.
 
*  High Positive Correlations between opinion of h1n1 risk, doctor recommendation of vaccines Vs whether the person really took the vaccine. Seems fairly obvious

*  Overall the data features seems to be positively correlating with the act of taking the vaccination, except with some rare differences

* Can mainly notice that there are many redundant features/sparsely correlated features which should be taken care of.

### Now Plotting the Heatmap of all the Categorical features which were label encoded for a better idea

In [ ]:
datum = data[str_cols]
corr = datum.corr()
g = sns.heatmap(corr,  vmax=.3, center=0,
            square=True, linewidths=1, cbar_kws={"shrink": .5}, annot=True, fmt='.2f', cmap='PuOr')
sns.despine()
g.figure.set_size_inches(10,10)
    
plt.show()

### Abundance of plots below!

* #### Pairwise Correlation plots of all the relevant and majorly significant features are given below. 
* #### Many features which are neutral and of no 'suggestive' value to the predictions should be removed later on to avoid redundancy.

In [ ]:
def vaccination_rate_plot(col, target, data, ax=None):
    """Stacked bar chart of vaccination rate for `target` against 
    `col`. 
    
    Args:
        col (string): column name of feature variable
        target (string): column name of target variable
        data (pandas DataFrame): dataframe that contains columns 
            `col` and `target`
        ax (matplotlib axes object, optional): matplotlib axes 
            object to attach plot to
    """
    counts = (data[[target, col]]
                  .groupby([target, col])
                  .size()
                  .unstack(target)
             )
    group_counts = counts.sum(axis='columns')
    props = counts.div(group_counts, axis='index')

    props.plot(kind="barh", stacked=True, ax=ax)
    ax.invert_yaxis()
    ax.legend().remove()

cols_to_plot = [
     'household_adults','opinion_h1n1_sick_from_vacc',
       'household_children', 'age_group', 'education', 'race',
       'sex', 'income_poverty', 'marital_status',
       'rent_or_own', 'employment_status', 'hhs_geo_region',
       'census_msa'
       
]

fig, ax = plt.subplots(
    len(cols_to_plot), 2, figsize=(9,len(cols_to_plot)*2.5)
)
for idx, col in enumerate(cols_to_plot):
    vaccination_rate_plot(
        col, 'h1n1_vaccine', data, ax=ax[idx, 0]
    )
    vaccination_rate_plot(
        col, 'seasonal_vaccine', data, ax=ax[idx, 1]
    )
    
ax[0, 0].legend(
    loc='lower center', bbox_to_anchor=(0.5, 1.05), title='h1n1_vaccine'
)
ax[0, 1].legend(
    loc='lower center', bbox_to_anchor=(0.5, 1.05), title='seasonal_vaccine'
)
fig.tight_layout()

* Census MSA, Race or Geographic Region don't seem to be suggestive and are kind of neutral. Might not be that helpful
* Employment status, Living on Rent and Marital Status maybe relevant for H1N1, but not as such for Seasonal
* Since there are a majority of people who have taken Seasonal compared to H1N1 vaccine, we might as well focus on the ones relevant to seasonal more, as there is a clearer distinction.

### More Plots!

In [ ]:
## The gender distribution of the data
sns.factorplot('sex',data=data,kind='count',size = 3.5)

## The number of people in each category of concern
sns.factorplot('h1n1_concern',kind = 'count',data = data)

In [ ]:
sns.factorplot('h1n1_knowledge',kind = 'count',data = data)
sns.factorplot('hhs_geo_region',kind = 'count',data = data)

In [ ]:
sns.factorplot('race',kind = 'count',data = data)
sns.factorplot('education',kind = 'count',data = data)

### Even More...

In [ ]:
df = data[data["seasonal_vaccine"] == 1].groupby(data['hhs_geo_region']).sum()
plt.title('Geographic region')
plt.plot(df['seasonal_vaccine'])
plt.show()
### people living in geographic regions 1,3,6 are more likely to get vaccinated than the rest

In [ ]:
df = data[data["h1n1_vaccine"] == 1].groupby(data['hhs_geo_region']).sum()
plt.title('Geographic region')
plt.plot(df['h1n1_vaccine'])
plt.show()

In [ ]:
df = data[data["h1n1_vaccine"] == 1].groupby(data['education']).sum()
plt.title('Education')
plt.plot(df['h1n1_vaccine'])
plt.show()

### Try to Infer what features are insignificant from the above plots

# 4. Feature Selection, Extraction & Engineering <a id="fe"></a> 

### Creating a new feature 'cleanliness' which combines the behaviour's

In [ ]:
data['cleanliness'] =  data['behavioral_antiviral_meds']+ data['behavioral_avoidance']+\
                        data['behavioral_face_mask']+data['behavioral_wash_hands']+\
                       data['behavioral_large_gatherings'] + data['behavioral_outside_home']+\
                       data['behavioral_touch_face']

In [ ]:
data['cleanliness']

In [ ]:
sns.factorplot('cleanliness',kind = 'count',data = data)

Checking the total number of people who have taken the H1N1 vaccine

In [ ]:
print(data[data["h1n1_vaccine"]==1].count()[1])

### Creating Plots for the new feature for each vaccine to check the correlation

In [ ]:
df = data[data["h1n1_vaccine"] == 1].groupby(data['cleanliness']).sum()
plt.title('Cleanliness')
plt.plot(df['h1n1_vaccine'])
plt.show()

In [ ]:
df = data[data["seasonal_vaccine"] == 1].groupby(data['cleanliness']).sum()
plt.title('Cleanliness')
plt.plot(df['seasonal_vaccine'])
plt.show()

People with cleanliness 2 or above are more likely to get vaccinated than the rest

### Creating a new feature 'opinion' which combines the several 'opinion' columns

In [ ]:
data['opinion'] = data['opinion_h1n1_vacc_effective'] + data['opinion_h1n1_risk']+\
                  data['opinion_h1n1_sick_from_vacc'] + data['opinion_seas_vacc_effective']+\
                  data['opinion_seas_risk'] + data['opinion_seas_sick_from_vacc']

In [ ]:
sns.factorplot('opinion',kind = 'count',data = data,size=9)

Ooh! a normal distribution like plot is always good to see!

In [ ]:
df = data[data["h1n1_vaccine"] == 1].groupby(data['opinion']).sum()
plt.title('Opinion')
plt.plot(df['h1n1_vaccine'])
plt.show()

### Creating seperate opinions for both vaccine's

In [ ]:
data['opinion_h1n1'] = data['opinion_h1n1_vacc_effective'] + data['opinion_h1n1_risk']-\
                  data['opinion_h1n1_sick_from_vacc'] 
data['opinion_seasonal'] = data['opinion_seas_vacc_effective']+\
                  data['opinion_seas_risk'] - data['opinion_seas_sick_from_vacc']

### Engineering some more features!

In [ ]:
data['concern>=2'] = np.where(data['h1n1_concern']>=2,1,0) 
# Since those who have concern>=2 have a pronounced effect on vaccination

data['good_opinion_vacc'] = np.where(data['opinion_seas_vacc_effective'] == 3,1,0)
#If the opinion is 3 it means he/she has a good opinion and is more likely to take the vaccine

data['good_knowledge'] = np.where(data['h1n1_knowledge'] == 2,1,0)
#Same kind of logic as mentioned previously

In [ ]:
data['risk'] = np.where(data['opinion_h1n1_risk']>=4,1,0)

#data['sick'] = np.where(data['opinion_h1n1_sick_from_vacc'] == 3,1,0)

### Generating a column combining concern and knowledge, since they have seemingly good correlations!

data['concern_knowledge'] = data['h1n1_concern']+data['h1n1_knowledge']

## square of age group to subtely improve the correlation of that feature with the vaccine probabilty
data['a^2'] = data['age_group']*data['age_group']

### Checking everythig out!

In [ ]:
data.columns

In [ ]:
data.head()

### Removing all the redundant, irrelevant and non-contributory features

In [ ]:
##### Removing seemingly irrelevant features ######

data.drop(['race','child_under_6_months','opinion_h1n1_sick_from_vacc','opinion_seas_sick_from_vacc','household_adults','behavioral_antiviral_meds','behavioral_large_gatherings', 'behavioral_outside_home', 'behavioral_antiviral_meds','marital_status',
           'behavioral_avoidance','behavioral_face_mask','income_poverty','hhs_geo_region','employment_status','education','census_msa'],axis=1,inplace = True)

Y_label = data[['h1n1_vaccine','seasonal_vaccine']]

### Whisker plots

Below plot indicates that there are few outliers and mostly similar distributions for doctor reccomendation (highly correlated feature) and opinion (an engineered feature).

In [ ]:
ax = sns.boxplot(x="doctor_recc_seasonal", y="opinion_seasonal", data=data)

cleanliness (an engineered feature) vs the 'opinion seasonal' ( highly correlated feature)

In [ ]:
ax = sns.boxplot(x="cleanliness", y="opinion_seasonal", data=data)

Cleanliness and Opinion

In [ ]:
ax = sns.boxplot(x="cleanliness", y="opinion", data=data)

In [ ]:
data.shape

### A final Correlation Map with our selected features

In [ ]:
corr = data.corr()
g = sns.heatmap(corr,  vmax=.3, center=0,
            square=True, linewidths=1, cbar_kws={"shrink": .5}, annot=True, fmt='.2f', cmap='coolwarm')
sns.despine()
g.figure.set_size_inches(30,25)
    
plt.show()

### We now see that the current 26 features are very well/ atleast better correlated with the ground truth labels than the previous Map!!

In [ ]:
#### Appending Y labels to the end of the dataframe ####

data.drop(['respondent_id','h1n1_vaccine','seasonal_vaccine'],axis=1,inplace = True)
frames =[data,Y_label]
data = pd.concat(frames,axis=1)

In [ ]:
data.head()

In [ ]:
features = data.columns[:-2]
print(features)
labels = ['h1n1_vaccine', 'seasonal_vaccine']

# 5. Data Splitting and Test data processing <a id="split"></a> 

### Choosing 80%, 20% for train and validation

In [ ]:
train,test = train_test_split(data,test_size = 0.2,shuffle = True)
train_x,train_y = train[features],train[labels]
test_x,test_y = test[features],test[labels]

In [ ]:
train_x.head()

In [ ]:
train_y.head()

In [ ]:
train_x.columns

### A Utility function is defined below which performs the exact same preprocessing, encodings, feature engineering and other transformations (which was done on the train data) on the Test data given to us.

In [ ]:
def transform_test(Test_X):
    Test_X.drop(['respondent_id','health_insurance','employment_industry','employment_occupation'],axis=1,inplace = True)

    # All the NULL values are populated with the mode
            
    d = Test_X
    str_cols = d.select_dtypes(include = 'object').columns

    ### LabelEcoding all categorical types #####
    for col in Test_X.columns:
        if Test_X[col].isnull().sum() and Test_X[col].dtypes != 'object':
            Test_X[col].loc[(Test_X[col].isnull())] = Test_X[col].median()
    for col in Test_X.columns:
        if Test_X[col].isnull().sum() and Test_X[col].dtypes == 'object':
            Test_X[col].loc[(Test_X[col].isnull())] = Test_X[col].mode().max()
    LE = LabelEncoder()
    for col in str_cols:
        Test_X[col] = LE.fit_transform(Test_X[col]) # Converts to int64
            
    data = d
    ### Synthesizing two new features cleanliness level of the individual and opinion of vaccine ####
    data['opinion'] = data['opinion_h1n1_vacc_effective'] + data['opinion_h1n1_risk']+\
                  data['opinion_h1n1_sick_from_vacc'] + data['opinion_seas_vacc_effective']+\
                  data['opinion_seas_risk'] + data['opinion_seas_sick_from_vacc']
    data['cleanliness'] =  data['behavioral_antiviral_meds']+ data['behavioral_avoidance']+\
                        data['behavioral_face_mask']+data['behavioral_wash_hands']+\
                       data['behavioral_large_gatherings'] + data['behavioral_outside_home']+\
                       data['behavioral_touch_face']
    data['opinion_h1n1'] = data['opinion_h1n1_vacc_effective'] + data['opinion_h1n1_risk']-\
                      data['opinion_h1n1_sick_from_vacc'] 
    data['opinion_seasonal'] = data['opinion_seas_vacc_effective']+\
                      data['opinion_seas_risk'] - data['opinion_seas_sick_from_vacc']

    data['concern>=2'] = np.where(data['h1n1_concern']>=2,1,0)
    data['good_opinion_vacc'] = np.where(data['opinion_seas_vacc_effective'] == 3,1,0) # 5 before
    data['good_knowledge'] = np.where(data['h1n1_knowledge'] == 2,1,0)
    data['risk'] = np.where(data['opinion_h1n1_risk']>=4,1,0)
    data['concern_knowledge'] = data['h1n1_concern']+data['h1n1_knowledge']
    data['a^2'] = data['age_group']*data['age_group']
    ###### Dropping other features #########
    data.drop(['race','child_under_6_months','opinion_h1n1_sick_from_vacc','opinion_seas_sick_from_vacc','household_adults','behavioral_antiviral_meds','behavioral_large_gatherings', 'behavioral_outside_home', 'behavioral_antiviral_meds','marital_status',
           'behavioral_avoidance','behavioral_face_mask','income_poverty','hhs_geo_region','employment_status','education','census_msa'],axis=1,inplace = True)
    Test_X = data
    return Test_X

Loading and transforming test data

In [ ]:
Test_X = pd.read_csv(r'/kaggle/input/flu-shot-prediction/test_set_features.csv')
Test_X.shape

In [ ]:
Test_X = transform_test(Test_X)

In [ ]:
Test_X.isnull().sum()

Similarity Check!!

In [ ]:
Test_X.columns

In [ ]:
train_x.columns

In [ ]:
Test_X.head()

# 6. Analyzing Several Models on the Dataset <a id="models"></a> 

In [ ]:
##### Model Generation #####

* ### **Remember that our task is not a Multi'Class' Classification problem, but instead a Multi'Label' Classification problem.**
* ### **Thus I have used the Binary Relevance Classifier to train a model to individually predict for each 'h1n1_vaccine' and for 'seasonal_vaccine' separately and later combine them to calculate the overall accuracy** 

## 6.1 Ensembling with 20 Classifiers and Linear models <a id="ensemble"></a> 

In [ ]:
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
MLA = [
    #Ensemble Methods
    ensemble.AdaBoostClassifier(),
    ensemble.BaggingClassifier(),
    ensemble.ExtraTreesClassifier(),
    ensemble.GradientBoostingClassifier(),
    ensemble.RandomForestClassifier(),

    #Gaussian Processes
    #gaussian_process.GaussianProcessClassifier(),
    
    #GLM
    linear_model.LogisticRegressionCV(),
    linear_model.PassiveAggressiveClassifier(),
    linear_model.RidgeClassifierCV(),
    linear_model.SGDClassifier(),
    linear_model.Perceptron(),
    
    #Navies Bayes
    naive_bayes.BernoulliNB(),
    naive_bayes.GaussianNB(),
    
    #Nearest Neighbor
    neighbors.KNeighborsClassifier(),
    
    #SVM
    #svm.SVC(probability=True),
    #svm.NuSVC(probability=True),
    svm.LinearSVC(),
    
    #Trees    
    tree.DecisionTreeClassifier(),
    tree.ExtraTreeClassifier(),
    
    #Discriminant Analysis
    discriminant_analysis.LinearDiscriminantAnalysis(),
    discriminant_analysis.QuadraticDiscriminantAnalysis(),

    
    #xgboost: http://xgboost.readthedocs.io/en/latest/model.html
    XGBClassifier()    
    ]



#split dataset in cross-validation with this splitter class: http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.ShuffleSplit.html#sklearn.model_selection.ShuffleSplit
#note: this is an alternative to train_test_split
cv_split = model_selection.ShuffleSplit(n_splits = 10, test_size = .3, train_size = .6, random_state = 0 ) # run model 10x with 60/30 split intentionally leaving out 10%

#create table to compare MLA metrics
MLA_columns = ['MLA Name', 'MLA Parameters', 'MLA Test Accuracy Mean', 'MLA Test Accuracy 3*STD' ,'MLA Time']
MLA_compare = pd.DataFrame(columns = MLA_columns)

#create table to compare MLA predictions
MLA_predict = data['h1n1_vaccine']

#index through MLA and save performance to table
row_index = 0
data1 = data.copy()
for alg in MLA:
    data = data1
    #set name and parameters
    MLA_name = alg.__class__.__name__
    print('Executing ',MLA_name)
    MLA_compare.loc[row_index, 'MLA Name'] = MLA_name
    MLA_compare.loc[row_index, 'MLA Parameters'] = str(alg.get_params())
    
    #score model with cross validation: http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html#sklearn.model_selection.cross_validate
    cv_results = model_selection.cross_validate(alg, data[features], data['h1n1_vaccine'], cv  = cv_split)
    #print(cv_results.keys())
    MLA_compare.loc[row_index, 'MLA Time'] = cv_results['fit_time'].mean()
    #MLA_compare.loc[row_index, 'MLA Train Accuracy Mean'] = cv_results['train_score'].mean()
    MLA_compare.loc[row_index, 'MLA Test Accuracy Mean'] = cv_results['test_score'].mean()   
    #if this is a non-bias random sample, then +/-3 standard deviations (std) from the mean, should statistically capture 99.7% of the subsets
    MLA_compare.loc[row_index, 'MLA Test Accuracy 3*STD'] = cv_results['test_score'].std()*3   #let's know the worst that can happen!
    
    #save MLA predictions - see section 6 for usage
    alg.fit(data[features], data['h1n1_vaccine'])
    MLA_predict[MLA_name] = alg.predict(data[features])
    row_index+=1

    
#print and sort table: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sort_values.html
MLA_compare.sort_values(by = ['MLA Test Accuracy Mean'], ascending = False, inplace = True)
MLA_compare
#MLA_predict


In [ ]:
#barplot using https://seaborn.pydata.org/generated/seaborn.barplot.html
sns.barplot(x='MLA Test Accuracy Mean', y = 'MLA Name', data = MLA_compare, color = 'm')

#prettify using pyplot: https://matplotlib.org/api/pyplot_api.html
plt.title('Machine Learning Algorithm Accuracy Score \n')
plt.xlabel('Accuracy Score (%)')
plt.ylabel('Algorithm')

In [ ]:
#### Utility Function to print the accuracy of the model ###

def find_acc(clf,X,truth,s):
    pred = clf.predict_proba(X.values)
    pred = pred.toarray()
    accuracy = roc_auc_score(truth,pred)
    print(s+" Accuracy is : ",accuracy*100)

<a id="hpo"></a> 
## 6.2 Trying out Hyper Parameter Optimization (HPO) on some Select Models. 

Tried out HPO with 

    1. Grid Search CV
    2. Randomized Search CV

### Check out the model and technique of HPO used in the comments given in each cell

In [ ]:
#### Random Forest Classifier with Grid Search CV for HPO #####

from sklearn.model_selection import GridSearchCV
grid_1 = {      "n_estimators"      : [100,200,500],
               "criterion"         : ["gini", "entropy"],
               "max_features"      : ['sqrt','log2',0.2,0.5,0.8],
               "max_depth"         : [3,4,6,10],
               "min_samples_split" : [2, 5, 20,50] }
RF=RandomForestClassifier()
gs = GridSearchCV(RF, grid_1, n_jobs=-1, cv=2,verbose = True)

In [ ]:
### Training & Performance ###

clf = BinaryRelevance(classifier=gs, require_dense=[True,True])
clf.fit(train_x,train_y)
find_acc(clf,train_x,train_y,'Training')
find_acc(clf,test_x,test_y,'Cross Validation')

In [ ]:
#### Random Forest Classifier with Randomized Search CV for HPO #####

from sklearn.model_selection import RandomizedSearchCV
n_estimators = [100,200,500]
max_features = ['auto', 'sqrt']
max_depth = [15,20,25]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
rf = RF
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 20,\
                               cv = 3, verbose=2, random_state=42, n_jobs = -1)

In [ ]:
### Training & Performance ###
clf = BinaryRelevance(classifier=rf_random, require_dense=[True,True])
clf.fit(train_x,train_y)
find_acc(clf,train_x,train_y,'Training')
find_acc(clf,test_x,test_y,'Cross Validation')

### A sample NN model for reference

In [ ]:
## Neural Network based Classifier on Keras sample ##

def NN_classifier(input_dim,output_dim):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(25,input_dim=input_dim,activation = 'relu'),
        tf.keras.layers.Dense(128,activation = 'relu'),
        tf.keras.layers.Dense(64,activation = 'relu'),
        tf.keras.layers.Dense(output_dim,activation = 'softmax')
    ])
    model.compile(loss = 'binary_crossentropy',optimizer = 'adam',metrics = ['accuracy'])
    return model

In [ ]:
#### To use NN Model ####

# KERAS_PARAMS = dict(epochs=100, batch_size=50, verbose=True)
# clf = BinaryRelevance(classifier=Keras(NN_classifier, False, KERAS_PARAMS), require_dense=[True,True])
# clf.fit(train_x,train_y)

#### To use RF basic #####

# RandomForestClassifier(max_depth = 5,random_state = 42,verbose = True)

#### To use RSCV RF ####

# rf_random

#### To us GSCV RF ####

# gs

In [ ]:
from sklearn.model_selection import GridSearchCV
param2 = {
            'learning_rate':     [0.01,0.1],
            'max_depth':         [2,3,4],
            'lambda':            [1.0,1.5],
            'subsample':        [0.1,0.2,0.3],
            'colsample_bytree': [0.3,0.5,0.6],
            'min_split_loss' :  [0.01,0.2],
            'min_split_size' : [2,4,6]
        }
classifier = XGBClassifier(feature_names = features,verbose = False)
xgb = GridSearchCV(classifier, param2, n_jobs=-1, cv=2,verbose = True)

In [ ]:
### Training & Performance ###
clf = BinaryRelevance(classifier=xgb, require_dense=[True,True])
clf.fit(train_x,train_y)
find_acc(clf,train_x,train_y,'Training')
find_acc(clf,test_x,test_y,'Cross Validation')

In [ ]:
#### Random Forest Classifier with Randomized Search CV for HPO #####

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import RandomizedSearchCV
n_estimators = [100,200]
max_features = ['auto', 'sqrt']
max_depth = [2,3,7,5,15,20,25]
max_depth.append(None)
min_samples_split = [0.2,0.3,0.6,1.2,1.5,2.0,3.0,4.0]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

rf = XGBClassifier(feature_names = features,verbose = False)
etc = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 20,\
                               cv = 4, verbose=2, random_state=42, n_jobs = -1)

In [ ]:
### Training & Performance ###
clf = BinaryRelevance(classifier=etc, require_dense=[True,True])
clf.fit(train_x,train_y)
find_acc(clf,train_x,train_y,'Training')
find_acc(clf,test_x,test_y,'Cross Validation')

In [ ]:
cb_hps = {  
            'depth':              [2,3,4,5,7],
            'l2_leaf_reg':        [2,3,5,10],
            'random_seed':        [5,8,10],
            'colsample_bylevel': [0.3,0.5,0.6],
            'n_estimators':      [100,200]
        }

cb = CatBoostClassifier()
cbt = RandomizedSearchCV(estimator = cb, param_distributions = cb_hps, n_iter = 20,\
                               cv = 4, verbose=2, random_state=42, n_jobs = -1)

In [ ]:
### Training & Performance ###

clf = BinaryRelevance(classifier=cbt, require_dense=[True,True])
clf.fit(train_x,train_y)
find_acc(clf,train_x,train_y,'Training')
find_acc(clf,test_x,test_y,'Cross Validation')

In [ ]:
#### Training a binary relevance classifier for the two labels h1n1 vaccine and seasonal vaccine #####

classifier = CatBoostClassifier(
n_estimators = 300,depth = 5,l2_leaf_reg =0.5,
                                random_seed = 2,colsample_bylevel = 0.9,verbose = False)

In [ ]:
### Training & Performance ###

clf = BinaryRelevance(classifier=classifier, require_dense=[True,True])
clf.fit(train_x,train_y)
find_acc(clf,train_x,train_y,'Training')
find_acc(clf,test_x,test_y,'Cross Validation')

# 7. Final Notes & Submission <a id="final"></a> 

1. We see that our training accuracies are consistently within the range of 85% - 90%
2. Our validation accuracies are also close to the training accuracies which indicates that the model hasn't fallen prey to overfitting/bias
3. Feel free to tweak around with any of the values of the hyper-parameters or fiddle with the models. These were the best results that i could obtain with the given models
4. Disclaimer: On submission to the competition site the model may give varying accuracies.
5. I have tried to demonstrate as many different model performances to minimize your time in developing a better more accurate model!

### Preparing the submission file in the specified format as given in 'submission_format.csv'

In [ ]:
submission = clf.predict_proba(Test_X.values)
submission_df = pd.DataFrame.sparse.from_spmatrix(submission)

In [ ]:
trial = submission_df
trial['respondent_id'] = trial.index+26707 # Converting the default indices
trial['h1n1_vaccine'] = trial[0].astype('float64') # Data required to be in float64 
trial['seasonal_vaccine'] = trial[1].astype('float64')
trial.set_index('respondent_id') # Make index as respondent_id
trial.drop([0,1],axis=1,inplace=True) # Drop old index axis
submission_df = trial 

In [ ]:
submission_df # Check if all OK

In [ ]:
submission_df.to_csv('submission.csv',index = False)

## That's It!
## You've now seen so many techniques of Preprocessing, EDA, Feature Selection, Feature Engineering,Ensembling, Linear Models and HPO!!
## I hope this notebook has helped you out towards your competitive ML journey

## Kindly upvote if you found the content useful !

### P.S: It takes a lot of time to develop these notebooks error free for you to understand, motivation is highly appreciated :-P ;-) 